In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
%matplotlib inline
sns.set_style(style='whitegrid')

In [3]:
from keys import keys
from binance.client import Client

In [4]:
client = Client(api_key=keys.apiKey, api_secret=keys.secretKey)

In [5]:
"""
The various columns used by Binance
taken from:
https://python-binance.readthedocs.io/en/latest/binance.html#module-binance.client
"""

columns = ['Open Time', 'Open', 'High', 'Low', 'Close',
            'Volume', 'Close Time', 'Quote asset volume', 
            'n_trades', 'Taker buy base asset volume', 
            'Taker buy quote asset volume', 'Ignore']

In [94]:
"""
DOWNLOADS DATA FROM BINANCE IN BATCHES OF 500 items

INPUTS:
    n_iterations: how many batches of 500 items to download
    
    currency: one of the varios currencies, examples: 
                'BTCUSDT', 'LTCUSDT' ...etc
    interval: dictionary, example: interval_5min = { 
                                        'interval' : Client.KLINE_INTERVAL_5MINUTE,
                                        'interval_diff' : 300000 
                                    }
            interval_diff: the difference in millisec of of new_data[1][0] - new_data[0][0]
            (the difference in binance timeframes of misurations)
"""
def get_by_intervals(n_iterations, currencies, interval):
    
    #columns template
    columns = ['Open Time', 'Open', 'High', 'Low', 'Close',
            'Volume', 'Close Time', 'Quote asset volume', 
            'n_trades', 'Taker buy base asset volume', 
            'Taker buy quote asset volume', 'Ignore']
    
    #most recent timestamp to start downloading from
    starting_point = client.get_klines(symbol=currencies[0], 
                                  interval=interval['interval'],
                                  limit=1
                                )[-1][0]
    
    main_df = pd.DataFrame()
    
    
    for currency in currencies:
        currency_cols = []
        for col in columns:
            currency_cols.append(currency+' '+col)
        
        specific_currency_df = pd.DataFrame(columns=currency_cols)
        
        #downloading n iterations of a specific currency
        for i in range(0,n_iterations):
            if specific_currency_df.empty:
                new_data = client.get_klines(symbol=currency, 
                                              interval=interval['interval'],
                                              endTime = starting_point,
                                              limit=500
                                            )
            else:
                end_time = specific_currency_df[currency+' Open Time'
                                               ].iloc[0]
                new_data = client.get_klines(symbol=currency, 
                                              interval=interval['interval'],
                                              endTime=end_time,
                                              limit=500)
            #merging all the rows of a specific currency in one DataFrame
            new_data_df = pd.DataFrame(new_data, columns=currency_cols)
            specific_currency_df = pd.concat([new_data_df,specific_currency_df,
                                             ignore_index=True)
            
        #merging the specific currency created into the main_df were we keep all the data
        main_df = pd.concat([main_df, specific_currency_df],axis=1)
    return main_df

In [95]:
"""
interval_diff: new_data[1][0] - new_data[0][0]
"""

interval_1min = { 'interval' : Client.KLINE_INTERVAL_1MINUTE,
                     'interval_diff': 60000 }

interval_5min = { 'interval' : Client.KLINE_INTERVAL_5MINUTE,
                    'interval_diff': 300000 }

interval_15min = { 'interval' : Client.KLINE_INTERVAL_15MINUTE,
                     'interval_diff': 900000 }

In [96]:
"""
USED TO TEST IF TIMESTAMPS ARE ALIGNED AND THERE ARE NO SUDDEN JUMPS

if it doesn't output anything we're good
"""

def test_time_skip(i_init,i_end,df, currencies):
    for i in range(i_init,i_end):
        for currency in currencies:
            if (df[currency+' Open Time'].iloc[i+1] - df[
                    currency+' Open Time'].iloc[i] != 300000):
                print('Time difference: ',currency,(df[currency+' Open Time'
                                  ].iloc[i+1] - df[currency+' Open Time'].iloc[i]))
                print('time 1: ',df[currency+' Open Time'].iloc[i+1])
                print('time 2: ',df[currency+' Open Time'].iloc[i],i)

In [97]:
ltc_test = get_by_intervals(15, ['LTCUSDT'], interval_5min)

In [98]:
ltc_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7486 entries, 0 to 7485
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   LTCUSDT Open Time                     7486 non-null   object
 1   LTCUSDT Open                          7486 non-null   object
 2   LTCUSDT High                          7486 non-null   object
 3   LTCUSDT Low                           7486 non-null   object
 4   LTCUSDT Close                         7486 non-null   object
 5   LTCUSDT Volume                        7486 non-null   object
 6   LTCUSDT Close Time                    7486 non-null   object
 7   LTCUSDT Quote asset volume            7486 non-null   object
 8   LTCUSDT n_trades                      7486 non-null   object
 9   LTCUSDT Taker buy base asset volume   7486 non-null   object
 10  LTCUSDT Taker buy quote asset volume  7486 non-null   object
 11  LTCUSDT Ignore                

In [99]:
ltc_test.head()

,LTCUSDT Open Time,LTCUSDT Open,LTCUSDT High,LTCUSDT Low,LTCUSDT Close,LTCUSDT Volume,LTCUSDT Close Time,LTCUSDT Quote asset volume,LTCUSDT n_trades,LTCUSDT Taker buy base asset volume,LTCUSDT Taker buy quote asset volume,LTCUSDT Ignore
0,1580821200000,67.05000000,67.14000000,66.95000000,67.07000000,860.08592000,1580821499999,57655.39805810,134,287.85769000,19293.59709610,0
1,1580821500000,67.08000000,67.20000000,67.01000000,67.15000000,665.02369000,1580821799999,44643.74379970,94,322.94363000,21688.42126640,0
2,1580821800000,67.13000000,67.14000000,67.02000000,67.02000000,950.36778000,1580822099999,63754.06198800,88,522.14957000,35031.49311060,0
3,1580822100000,67.05000000,67.09000000,66.87000000,66.92000000,648.84824000,1580822399999,43471.05980400,102,158.37319000,10605.62205290,0
4,1580822400000,66.86000000,66.95000000,66.69000000,66.90000000,1810.77580000,1580822699999,120991.71131160,255,684.21559000,45711.92125680,0


In [100]:
#print(ltc_test['LTCUSDT Open Time'][0])
#print(ltc_test['BTCEUR Open Time'][0])

In [101]:
ltc_test[ ltc_test['LTCUSDT Open Time'].duplicated()]

,LTCUSDT Open Time,LTCUSDT Open,LTCUSDT High,LTCUSDT Low,LTCUSDT Close,LTCUSDT Volume,LTCUSDT Close Time,LTCUSDT Quote asset volume,LTCUSDT n_trades,LTCUSDT Taker buy base asset volume,LTCUSDT Taker buy quote asset volume,LTCUSDT Ignore


In [102]:
test_time_skip(0,7485,ltc_test,['LTCUSDT'])

Time difference:  LTCUSDT 3900000
time 1:  1581217200000
time 2:  1581213300000 1307
Time difference:  LTCUSDT 21300000
time 1:  1582133400000
time 2:  1582112100000 4291


In [15]:
"""
ltc_test[['Open','Close', 'High','Low', 'Volume', 'Quote asset volume', 
            'Taker buy base asset volume', 'Taker buy quote asset volume'
           ]] = ltc_test.columns.drop(''.astype(float)
                                      """

"\nltc_test[['Open','Close', 'High','Low', 'Volume', 'Quote asset volume', \n            'Taker buy base asset volume', 'Taker buy quote asset volume'\n           ]] = ltc_test.columns.drop(''.astype(float)\n                                      "

In [16]:
ltc_test['Close Time'] = ltc_test['Close Time'].apply(
    lambda x: datetime.fromtimestamp(int(x/1000)))
ltc_test['Open Time'] = ltc_test['Open Time'].apply(
    lambda x: datetime.fromtimestamp(int(x/1000)))

KeyError: 'Close Time'

In [17]:
ltc_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   LTCUSDT Open Time                     7500 non-null   object
 1   LTCUSDT Open                          7500 non-null   object
 2   LTCUSDT High                          7500 non-null   object
 3   LTCUSDT Low                           7500 non-null   object
 4   LTCUSDT Close                         7500 non-null   object
 5   LTCUSDT Volume                        7500 non-null   object
 6   LTCUSDT Close Time                    7500 non-null   object
 7   LTCUSDT Quote asset volume            7500 non-null   object
 8   LTCUSDT n_trades                      7500 non-null   object
 9   LTCUSDT Taker buy base asset volume   7500 non-null   object
 10  LTCUSDT Taker buy quote asset volume  7500 non-null   object
 11  LTCUSDT Ignore                

In [18]:
print('Data timeframe:')
print( 'From:  ',ltc_test['Close Time'].max())
print( 'To     ',ltc_test['Close Time'].min())

Data timeframe:


KeyError: 'Close Time'

In [19]:
plt.figure(figsize=(16,8))
ltc_test['Close'].iloc[:500].plot()

KeyError: 'Close'

<Figure size 1152x576 with 0 Axes>

In [36]:
new_data = client.get_klines(symbol='BTCEUR', 
    interval=Client.KLINE_INTERVAL_1MINUTE )

In [37]:
new_data[0]

[1583060100000,
 '7764.65000000',
 '7764.65000000',
 '7764.65000000',
 '7764.65000000',
 '0.00000000',
 1583060159999,
 '0.00000000',
 0,
 '0.00000000',
 '0.00000000',
 '0']

In [ ]:
second_new[0]

In [48]:
i=0
arr= pd.DataFrame(columns=columns)
for i in range(0,20):
    if i ==0:
        second_new = client.get_klines(symbol='LTCUSDT', 
            interval=Client.KLINE_INTERVAL_15MINUTE,
            endTime=new_data[0][0], limit=500)
    else:
        second_new = client.get_klines(symbol='LTCUSDT', 
            interval=Client.KLINE_INTERVAL_15MINUTE,
            endTime=(second_new[0][0] - 900000), limit=500)
    arr = pd.concat([pd.DataFrame(data=second_new, columns=columns),arr], ignore_index=True)

In [49]:
arr.reset_index()
arr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Open Time                     10000 non-null  object
 1   Open                          10000 non-null  object
 2   High                          10000 non-null  object
 3   Low                           10000 non-null  object
 4   Close                         10000 non-null  object
 5   Volume                        10000 non-null  object
 6   Close Time                    10000 non-null  object
 7   Quote asset volume            10000 non-null  object
 8   n_trades                      10000 non-null  object
 9   Taker buy base asset volume   10000 non-null  object
 10  Taker buy quote asset volume  10000 non-null  object
 11  Ignore                        10000 non-null  object
dtypes: object(12)
memory usage: 937.6+ KB


In [50]:
arr['Open Time'][0]

1574028900000

In [51]:
arr['Open Time'][500]

1574478900000

In [52]:
for i in range(0,9999):

    if ((arr['Open Time'].iloc[i+1] - arr['Open Time'].iloc[i]) != 900000):
        print('Time difference: ', arr['Open Time'].iloc[i+1] - arr['Open Time'].iloc[i])
        print('time 1: ',arr['Open Time'].iloc[i+1])
        print('time 2: ',arr['Open Time'].iloc[i],i)

Time difference:  8100000
time 1:  1574654400000
time 2:  1574646300000 686
Time difference:  4500000
time 1:  1581217200000
time 2:  1581212700000 7974
Time difference:  21600000
time 1:  1582133400000
time 2:  1582111800000 8969


In [53]:
arr[arr['Open Time'].duplicated()]

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,n_trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore


In [88]:
df = client.get_klines(symbol='LTCUSDT', 
            interval=Client.KLINE_INTERVAL_15MINUTE,)

In [89]:
df[0][0]

1582641900000

In [90]:
df2 = client.get_klines(symbol='LTCUSDT', 
            interval=Client.KLINE_INTERVAL_15MINUTE,
                       endTime=df[0][0])

In [91]:
df = pd.DataFrame(data=df, columns=columns)[1:]
df2 = pd.DataFrame(data=df2, columns=columns)
df_end = pd.concat([df2,df], ignore_index=True)
arr=df_end
arr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Open Time                     999 non-null    int64 
 1   Open                          999 non-null    object
 2   High                          999 non-null    object
 3   Low                           999 non-null    object
 4   Close                         999 non-null    object
 5   Volume                        999 non-null    object
 6   Close Time                    999 non-null    int64 
 7   Quote asset volume            999 non-null    object
 8   n_trades                      999 non-null    int64 
 9   Taker buy base asset volume   999 non-null    object
 10  Taker buy quote asset volume  999 non-null    object
 11  Ignore                        999 non-null    object
dtypes: int64(3), object(9)
memory usage: 93.8+ KB


In [92]:
for i in range(0,998):

    if ((arr['Open Time'].iloc[i+1] - arr['Open Time'].iloc[i]) != 900000):
        print('Time difference: ', arr['Open Time'].iloc[i+1] - arr['Open Time'].iloc[i])
        print('time 1: ',arr['Open Time'].iloc[i+1])
        print('time 2: ',arr['Open Time'].iloc[i],i)

In [93]:
df_end[ df_end['Open Time'].duplicated()]

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,n_trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
